<a href="https://colab.research.google.com/github/ffabi/VisualAnalysis/blob/main/M%C3%A9r%C3%A9si_aggregated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Processsing aggregated solar and weather data

In [188]:
# Download data
import os
import pandas as pd
os.environ['KAGGLE_USERNAME'] = "ffabian"
os.environ['KAGGLE_KEY'] = "a06083fe469934fbd0ec143ea1681c92"
!kaggle datasets download ramima/weather-dataset-in-antwerp-belgium -q
!kaggle datasets download fvcoppen/solarpanelspower -q
!unzip -q -u weather-dataset-in-antwerp-belgium.zip
!unzip -q -u solarpanelspower.zip

In [189]:
solar = pd.read_csv('PV_Elec_Gas3.csv', ',')
solar = solar.rename(columns = {"Unnamed: 0" : "date"})

# Adaptation, dataset changed
solar = solar.rename(columns = {"Cumulative_solar_power" : "cum_power"})
solar = solar.rename(columns = {"kWh electricity/day" : "Elec_kW"})
solar = solar.rename(columns = {"Gas/day" : "Gas_mxm"})

# create separate year, month, day values
solar["date"] = pd.DatetimeIndex(pd.to_datetime(solar["date"], format="%d/%m/%Y"))
solar['year'] = pd.to_numeric(solar['date'].dt.strftime('%Y'))
solar['month'] = pd.to_numeric(solar['date'].dt.strftime('%m'))
solar['day'] = pd.to_numeric(solar['date'].dt.strftime('%d'))

# calculate daily power
solar = solar.set_index("date")
solar2 = solar.shift(periods=1, freq='D', axis=0)
solar['cum_power_shift'] = solar2.loc[:,'cum_power']
solar['daily_energy'] = solar['cum_power'].values - solar['cum_power_shift'].values

solar = solar[(solar["year"] >= 2012)]

solar = solar.drop(['cum_power'], axis=1)
solar = solar.drop(['cum_power_shift'], axis=1)
solar = solar.reset_index()
# solar["date"] = pd.to_datetime(solar["date"])

solar["is_hot"] = solar["Gas_mxm"] <= 3 # todo: compare to and use weather data
solar["is_hot"] = solar["is_hot"].astype(int)
# mask = solar.applymap(type) != bool
# d = {True: 'TRUE', False: 'FALSE'}
# solar = solar.where(mask, solar.replace(d))

solar["consumption"] = solar["Elec_kW"] + solar["daily_energy"]

solar = solar.sort_values(by="date").reset_index(drop=True)

In [190]:
solar

,date,Elec_kW,Gas_mxm,year,month,day,daily_energy,is_hot,consumption
0,2012-01-01,16.0,13.6,2012,1,1,0.5,0,16.5
1,2012-01-02,18.8,10.6,2012,1,2,0.8,0,19.6
2,2012-01-03,15.5,11.7,2012,1,3,2.9,0,18.4
3,2012-01-04,19.3,19.3,2012,1,4,0.8,0,20.1
4,2012-01-05,6.9,10.6,2012,1,5,2.7,0,9.6
...,...,...,...,...,...,...,...,...,...
3232,2020-11-06,16.0,11.0,2020,11,6,8.0,0,24.0
3233,2020-11-07,13.0,13.0,2020,11,7,8.0,0,21.0
3234,2020-11-08,12.0,11.0,2020,11,8,8.0,0,20.0
3235,2020-11-09,14.0,10.0,2020,11,9,5.0,0,19.0


In [209]:
weather = pd.read_csv("weather_in_Antwerp.csv", ";")

weather = weather.rename(columns = {"Unnamed: 0" : "datetime", "barometer": "pressure"})

weather = weather[weather["temp"].notna()]

weather["hour"] = weather["clock"].str.split(":", n = 1, expand=True)[0].astype(int)
weather["minute"] = weather["clock"].str.split(":", n = 1, expand=True)[1].astype(int)
weather["datetime"] = pd.to_datetime(dict(year=weather["year"], month=weather["month"], day=weather["day"], hour=weather["hour"], minute=weather["minute"]))

weather["pressure"] = weather["pressure"].apply(lambda x: x.replace(" mbar", "") if isinstance(x, str) else x).astype(float)
weather["humidity"] = weather["humidity"].apply(lambda x: x.replace("%", "") if isinstance(x, str) else x).astype(float)
weather["temp"] = weather["temp"].apply(lambda x: x.replace("°C", "") if isinstance(x, str) else x).astype(float)
weather["wind"] = weather["wind"].apply(lambda x: x.replace(" km/h", "") if isinstance(x, str) else x)
weather["wind"] = weather["wind"].apply(lambda x: 0 if (isinstance(x, str) and x == "No wind") else x).astype(float)
weather["visibility"] = weather["visibility"].fillna("99\xa0km").apply(lambda x: x.replace("\xa0km", "") if isinstance(x, str) else x).astype(int)

import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

weather["is_clear"] = weather["weather"].str.lower().str.contains("(clear|sunny)", regex=True).astype(int)
weather["is_cloudy"] = weather["weather"].str.lower().str.contains("(cloud|overcast|fog)", regex=True).astype(int)
weather["date"] = weather["datetime"].dt.date

In [210]:
!pip3 install -q suntime
import datetime
from suntime import Sun, SunTimeException
import pytz

def sun_is_up(date_and_time):
    date_and_time = date_and_time.replace(tzinfo=pytz.utc)
    latitude = 51.21
    longitude = 4.42
    sun = Sun(latitude, longitude)
    date = date_and_time.date()

    try:
        sunrise = sun.get_local_sunrise_time(date)
        sunset = sun.get_local_sunset_time(date)
    except SunTimeException as e:
        print("Error: {0}.".format(e))

    return sunrise <= date_and_time <= sunset

In [211]:
# use weather data only after sunrise and before sunset
weather = weather[list(map(sun_is_up, weather["datetime"]))]

In [212]:
daily_weather = weather.resample('D', on='datetime').mean().reset_index()
# daily_weather = weather.resample('D', on='datetime').agg({'temp': "mean", 'weather': 'median'}) # todo aggregate other columns
daily_weather = daily_weather.rename(columns = {"datetime":"date"})
daily_weather

,date,temp,wind,humidity,pressure,visibility,year,month,day,hour,minute,is_clear,is_cloudy
0,2012-01-01,12.533333,19.333333,90.000000,1009.733333,74.4000,2012.0,1.0,1.0,11.200000,36.0,0.0000,1.0000
1,2012-01-02,8.066667,16.600000,82.133333,1008.000000,99.0000,2012.0,1.0,2.0,10.800000,36.0,0.0000,1.0000
2,2012-01-03,9.312500,41.000000,82.687500,1005.000000,58.3750,2012.0,1.0,3.0,11.000000,35.0,0.7500,0.2500
3,2012-01-04,7.375000,25.750000,67.875000,1017.062500,99.0000,2012.0,1.0,4.0,11.000000,35.0,0.1875,0.8125
4,2012-01-05,8.812500,41.062500,74.750000,993.875000,81.4375,2012.0,1.0,5.0,11.125000,35.0,0.2500,0.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,2019-12-26,3.625000,10.312500,90.312500,1022.812500,21.6875,2019.0,12.0,26.0,11.000000,35.0,0.7500,0.2500
2917,2019-12-27,6.800000,10.133333,92.066667,1027.533333,5.4000,2019.0,12.0,27.0,11.066667,34.0,0.2000,0.8000
2918,2019-12-28,3.687500,6.625000,87.875000,1038.937500,19.9375,2019.0,12.0,28.0,11.000000,35.0,0.4375,0.5625
2919,2019-12-29,1.875000,6.125000,82.437500,1036.500000,15.0000,2019.0,12.0,29.0,11.000000,35.0,1.0000,0.0000


In [213]:

# no weather data on 31th of each month, so drop it from solar as well
dropped_solar = solar[solar["day"] != 31]
dropped_solar = dropped_solar.drop(columns = ["year", "month", "day"], axis=1)
merged = pd.merge(daily_weather, dropped_solar, on="date")

In [214]:
merged["unix_time"] = merged["date"].astype(int) / 10**9

In [215]:
merged

,date,temp,wind,humidity,pressure,visibility,year,month,day,hour,minute,is_clear,is_cloudy,Elec_kW,Gas_mxm,daily_energy,is_hot,consumption,unix_time
0,2012-01-01,12.533333,19.333333,90.000000,1009.733333,74.4000,2012.0,1.0,1.0,11.200000,36.0,0.0000,1.0000,16.0,13.6,0.5,0,16.5,1.325376e+09
1,2012-01-02,8.066667,16.600000,82.133333,1008.000000,99.0000,2012.0,1.0,2.0,10.800000,36.0,0.0000,1.0000,18.8,10.6,0.8,0,19.6,1.325462e+09
2,2012-01-03,9.312500,41.000000,82.687500,1005.000000,58.3750,2012.0,1.0,3.0,11.000000,35.0,0.7500,0.2500,15.5,11.7,2.9,0,18.4,1.325549e+09
3,2012-01-04,7.375000,25.750000,67.875000,1017.062500,99.0000,2012.0,1.0,4.0,11.000000,35.0,0.1875,0.8125,19.3,19.3,0.8,0,20.1,1.325635e+09
4,2012-01-05,8.812500,41.062500,74.750000,993.875000,81.4375,2012.0,1.0,5.0,11.125000,35.0,0.2500,0.7500,6.9,10.6,2.7,0,9.6,1.325722e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861,2019-12-26,3.625000,10.312500,90.312500,1022.812500,21.6875,2019.0,12.0,26.0,11.000000,35.0,0.7500,0.2500,17.0,12.0,1.0,0,18.0,1.577318e+09
2862,2019-12-27,6.800000,10.133333,92.066667,1027.533333,5.4000,2019.0,12.0,27.0,11.066667,34.0,0.2000,0.8000,11.0,14.0,2.0,0,13.0,1.577405e+09
2863,2019-12-28,3.687500,6.625000,87.875000,1038.937500,19.9375,2019.0,12.0,28.0,11.000000,35.0,0.4375,0.5625,16.0,13.0,1.0,0,17.0,1.577491e+09
2864,2019-12-29,1.875000,6.125000,82.437500,1036.500000,15.0000,2019.0,12.0,29.0,11.000000,35.0,1.0000,0.0000,18.0,15.0,3.0,0,21.0,1.577578e+09


In [216]:
merged.describe()

,temp,wind,humidity,pressure,visibility,year,month,day,hour,minute,is_clear,is_cloudy,Elec_kW,Gas_mxm,daily_energy,is_hot,consumption,unix_time
count,2865.000000,2865.000000,2860.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2865.000000,2866.000000,2866.000000,2866.000000,2866.000000,2866.000000,2.866000e+03
mean,12.107697,14.087403,72.950319,1015.614195,62.775884,2015.500175,6.523560,15.430366,11.128527,35.129273,0.337318,0.656315,4.451256,8.448604,11.011235,0.316469,15.462491,1.451535e+09
std,6.346414,6.074569,10.980807,9.726810,27.556905,2.291307,3.446248,8.623170,0.257671,0.720266,0.234970,0.234297,9.868791,6.430979,8.216856,0.465179,3.491306,7.289100e+07
min,-10.526316,2.555556,31.424242,974.625000,0.000000,2012.000000,1.000000,1.000000,6.384615,30.800000,0.000000,0.000000,-24.000000,0.000000,0.000000,0.000000,-2.000000,1.325376e+09
25%,7.411765,9.526316,64.887821,1009.928571,46.052632,2014.000000,4.000000,8.000000,11.000000,35.000000,0.153846,0.500000,-3.000000,2.000000,4.000000,0.000000,13.100000,1.388405e+09
50%,11.896552,13.461538,73.272222,1016.100000,66.909091,2016.000000,7.000000,15.000000,11.156250,35.000000,0.304348,0.687500,6.000000,8.000000,9.700000,0.000000,15.900000,1.451520e+09
75%,16.967742,17.636364,81.398396,1022.105263,85.166667,2017.000000,10.000000,23.000000,11.258065,35.517241,0.484848,0.840000,13.000000,14.000000,17.000000,1.000000,17.500000,1.514570e+09
max,31.466667,48.727273,100.000000,1045.125000,99.000000,2019.000000,12.000000,30.000000,14.272727,41.176471,1.000000,1.000000,34.000000,29.000000,34.000000,1.000000,47.200000,1.577664e+09


In [217]:
#@title
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_matrix(merged, 
                        dimensions=["date","daily_energy", "consumption", "Gas_mxm", "temp", "wind", "visibility", "is_clear", "is_cloudy", "is_hot"], 
                        template="plotly_white",
                        color="unix_time"
                        )

fig.show()

In [218]:
#@title
title = "Histogram of temperature colored by is_hot"
fig = px.histogram(merged, x="temp", nbins = 80, 
                   color="is_hot", 
                   template="plotly_white", title=title)
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")

In [219]:
#@title
title = "Scatter plot of produced energy vs temperature"
fig = px.scatter(merged,
                 x="temp", y="daily_energy",
                 template="plotly_white",
                 trendline="lowess",
                #  color="unix_time",
                 opacity=0.2,
                 )


fig.show()
fig.write_html(title.replace(" ", "_") + ".html")

In [220]:
#@title
title = "Scatter plot of gas usage vs temperature"
fig = px.scatter(merged,
                 x="temp", y="Gas_mxm",
                 template="plotly_white",
                 trendline="lowess",
                #  color="unix_time",
                 opacity=0.2
                 )

fig.show()
fig.write_html(title.replace(" ", "_") + ".html")

In [221]:
#@title
title = "Scatter plot of energy consumption vs temperature"
fig = px.scatter(merged,
                 x="temp", y="consumption",
                 template="plotly_white",
                 trendline="lowess",
                #  color="unix_time",
                 opacity=0.2,
                 )
fig.show()
fig.write_html(title.replace(" ", "_") + ".html")

In [229]:
#@title
title = "Scatter plot of produced energy vs temperature"
fig = px.scatter(merged,
                 x="is_clear", y="is_cloudy",
                 template="plotly_white",
                #  trendline="lowess",
                #  color="unix_time",
                 opacity=0.1,
                 )


fig.show()
fig.write_html(title.replace(" ", "_") + ".html")

In [227]:
weather["weather"].value_counts().head(20)

Passing clouds.                 21619
Scattered clouds.                8398
Sunny.                           8046
Partly sunny.                    7472
Clear.                           5381
Fog.                             4187
Broken clouds.                   4040
Light rain. Broken clouds.        999
Light rain. Partly sunny.         825
Sprinkles. Broken clouds.         636
Ice fog.                          560
Overcast.                         532
Drizzle. Broken clouds.           497
Drizzle. Fog.                     476
Partly cloudy.                    369
Drizzle. Partly sunny.            352
Sprinkles. Scattered clouds.      319
Light rain. Fog.                  302
Light rain. Passing clouds.       298
Sprinkles. Partly sunny.          298
Name: weather, dtype: int64